In [ ]:
import numpy as np
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB 


class AutoML:
    def __init__(self, initial_temp=100, cooling_rate=0.99, max_iterations=100, min_training_time=3600):
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.min_training_time = max(min_training_time, 3600)

        self.algorithms = {
            'DecisionTreeClassifier': {
                'class': DecisionTreeClassifier,
                'parameters': ['max_depth', 'min_samples_split'],
                'ranges': [(1, 20), (2, 20)]
            },
            'LogisticRegression': {
                'class': LogisticRegression,
                'parameters': ['C', 'penalty'],
                'ranges': [(0.01, 10), ['l2', 'l1']]
            },
            'SVC': {
                'class': SVC,
                'parameters': ['C', 'gamma'],
                'ranges': [(0.01, 10), (0.01, 1)]
            },
            'RandomForestClassifier': {
                'class': RandomForestClassifier,
                'parameters': ['n_estimators', 'max_depth'],
                'ranges': [(10, 100), (1, 20)]
            },
            'GradientBoostingClassifier': {
                'class': GradientBoostingClassifier,
                'parameters': ['n_estimators', 'learning_rate'],
                'ranges': [(10, 100), (0.01, 0.3)]
            },
            'MLPClassifier': {
                'class': MLPClassifier,
                'parameters': ['alpha', 'learning_rate_init'],
                'ranges': [(0.0001, 0.1), (0.0001, 0.1)]
            },
            'GaussianNB': { 
                'class': GaussianNB,
                'parameters': [],  
                'ranges': []      
            }
        }

        self.best_solution = None
        self.best_score = 0
        self.model = None

    def eval(self, model, X, y):
        scores = cross_val_score(model, X, y, cv=5)
        print('scores=' + str(np.mean(scores)))
        return np.mean(scores)

    def generate_neighborhood(self, current_solution):
        new_solution = current_solution.copy()
        algorithm_name = np.random.choice(list(self.algorithms.keys()))
        new_solution[0] = algorithm_name

        algorithm_info = self.algorithms[algorithm_name]
        for i, parameter in enumerate(algorithm_info['parameters']):
            low, high = algorithm_info['ranges'][i]
            
            if isinstance(high, list):  
                new_solution[i + 1] = np.random.choice(high)
            elif isinstance(high, str):  
                new_solution[i + 1] = high  
            elif isinstance(low, int) and isinstance(high, int): 
                new_solution[i + 1] = np.random.randint(low, high)
            else:  
                new_solution[i + 1] = np.random.uniform(low, high)

        print(f"Neighborhood algorithm: {algorithm_name}, parameters: {new_solution[1:]}")
        return new_solution

    def create_model(self, solution):
        algorithm_name = solution[0]
        hyperparameters = solution[1:]
        algorithm_class = self.algorithms[algorithm_name]['class']
        if algorithm_name == 'LogisticRegression':
            return algorithm_class(C=hyperparameters[0], penalty=hyperparameters[1], solver='liblinear')
        elif algorithm_name == 'MLPClassifier':
            return algorithm_class(alpha=hyperparameters[0], learning_rate_init=hyperparameters[1])
        elif algorithm_name == 'GaussianNB':
            return algorithm_class()  # No hyperparameters to pass for GaussianNB
        else:
            return algorithm_class(**{param: int(value) if param in ['max_depth', 'n_estimators', 'min_samples_split'] else value for param, value in zip(self.algorithms[algorithm_name]['parameters'], hyperparameters)})

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.simulated_annealing()

    def predict(self, X):
        if self.model is None:
            raise ValueError("The model has not been fit yet. Please call the fit method first.")
        return self.model.predict(X)

    def simulated_annealing(self):
        start_time = time.time()  # Track the start time of the process
        
        current_model = [np.random.choice(list(self.algorithms.keys()))]
        if current_model[0] == 'RandomForestClassifier' or current_model[0] == 'DecisionTreeClassifier':
            current_hyperparameters = [
                np.random.randint(*self.algorithms[current_model[0]]['ranges'][i])
                if self.algorithms[current_model[0]]['parameters'][i] in ['n_estimators', 'max_depth', 'min_samples_split']
                else np.random.uniform(*self.algorithms[current_model[0]]['ranges'][i])
                for i in range(len(self.algorithms[current_model[0]]['parameters']))
            ]
        elif current_model[0] == 'GradientBoostingClassifier':
            current_hyperparameters = [
                np.random.randint(*self.algorithms[current_model[0]]['ranges'][i]) if self.algorithms[current_model[0]]['parameters'][i] in ['n_estimators']
                else np.random.uniform(*self.algorithms[current_model[0]]['ranges'][i])
                for i in range(len(self.algorithms[current_model[0]]['parameters']))
            ]
        elif current_model[0] == 'SVC':
            current_hyperparameters = [
                np.random.uniform(*self.algorithms[current_model[0]]['ranges'][i]) if self.algorithms[current_model[0]]['parameters'][i] != 'penalty'
                else np.random.choice(self.algorithms[current_model[0]]['ranges'][i])
                for i in range(len(self.algorithms[current_model[0]]['parameters']))
            ]
        elif current_model[0] == 'LogisticRegression':
            current_hyperparameters = [
                np.random.uniform(*self.algorithms[current_model[0]]['ranges'][i])
                if self.algorithms[current_model[0]]['parameters'][i] != 'penalty'
                else np.random.choice(self.algorithms[current_model[0]]['ranges'][i])
                for i in range(len(self.algorithms[current_model[0]]['parameters']))
            ]
        elif current_model[0] == 'GaussianNB':  # No hyperparameters to generate for GaussianNB
            current_hyperparameters = []
        else:
            current_hyperparameters = [
                np.random.uniform(*self.algorithms[current_model[0]]['ranges'][i])
                for i in range(len(self.algorithms[current_model[0]]['parameters']))
            ]
    
        current_solution = current_model + current_hyperparameters
    
        print(f"Initial model: {current_model[0]}")
        print(f"Initial parameters: {current_hyperparameters}")
    
        current_score = self.eval(self.create_model(current_solution), self.X, self.y)
        best_solution = current_solution
        best_score = current_score
    
        temperature = self.initial_temp
    
        while time.time() - start_time < self.min_training_time:
            for _ in range(100):
                new_solution = self.generate_neighborhood(current_solution)
                new_score = self.eval(self.create_model(new_solution), self.X, self.y)
    
                if new_score > current_score:
                    current_solution = new_solution
                    current_score = new_score
                    if new_score > best_score:
                        best_solution = new_solution
                        best_score = new_score
                else:
                    acceptance_probability = np.exp((new_score - current_score) / temperature)
                    if np.random.rand() < acceptance_probability:
                        current_solution = new_solution
                        current_score = new_score
    
            temperature *= self.cooling_rate
    
        self.best_solution = best_solution
        self.best_score = best_score
        self.model = self.create_model(best_solution)


X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)
automl = AutoML(min_training_time=3600)  # Set minimum training time to 1 hour
automl.fit(X, y)
predictions = automl.predict(X)


Initial model: LogisticRegression
Initial parameters: [6.3551245858199, 'l2']
scores=0.821
Neighborhood algorithm: MLPClassifier, parameters: [0.08738890087656082, 0.09480614989204633]
scores=0.933
Neighborhood algorithm: MLPClassifier, parameters: [0.07658556882298888, 0.024849032484196357]
scores=0.9369999999999999
Neighborhood algorithm: RandomForestClassifier, parameters: [90, 12]
scores=0.9319999999999998
Neighborhood algorithm: GradientBoostingClassifier, parameters: [39, 0.23318105874789755]
scores=0.916
Neighborhood algorithm: RandomForestClassifier, parameters: [91, 8]
scores=0.9259999999999999
Neighborhood algorithm: SVC, parameters: [5.9412051058605435, 0.5154615524121159]
scores=0.8969999999999999
Neighborhood algorithm: GradientBoostingClassifier, parameters: [67, 0.25618734090369844]
scores=0.922
Neighborhood algorithm: RandomForestClassifier, parameters: [78, 3]
scores=0.883
Neighborhood algorithm: RandomForestClassifier, parameters: [23, 5]
scores=0.917
Neighborhood alg

/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


scores=0.932
Neighborhood algorithm: LogisticRegression, parameters: [9.67374594347519, 'l1']
scores=0.82
Neighborhood algorithm: SVC, parameters: [7.517504939113297, 0.011058461288946657]
scores=0.908
Neighborhood algorithm: MLPClassifier, parameters: [0.06596337322403992, 0.023053962384189455]
scores=0.931
Neighborhood algorithm: MLPClassifier, parameters: [0.012924765444343098, 0.039737140275139235]
scores=0.93
Neighborhood algorithm: DecisionTreeClassifier, parameters: [1, 19]
scores=0.719
Neighborhood algorithm: LogisticRegression, parameters: [7.337639196952333, 'l1']
scores=0.82
Neighborhood algorithm: DecisionTreeClassifier, parameters: [10, 15]
scores=0.8939999999999999
Neighborhood algorithm: GaussianNB, parameters: [10, 15]
scores=0.8470000000000001
Neighborhood algorithm: GaussianNB, parameters: [10, 15]
scores=0.8470000000000001
Neighborhood algorithm: LogisticRegression, parameters: [0.8265335454662427, 'l1']
scores=0.821
Neighborhood algorithm: DecisionTreeClassifier, pa

/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

scores=0.943
Neighborhood algorithm: SVC, parameters: [3.036224723199715, 0.9004762608287062]
scores=0.825
Neighborhood algorithm: LogisticRegression, parameters: [1.7827947428791795, 'l1']
scores=0.82
Neighborhood algorithm: LogisticRegression, parameters: [8.552601124351014, 'l1']
scores=0.82
Neighborhood algorithm: GradientBoostingClassifier, parameters: [43, 0.20929295171057963]
scores=0.9189999999999999
Neighborhood algorithm: RandomForestClassifier, parameters: [90, 10]
scores=0.9289999999999999
Neighborhood algorithm: GaussianNB, parameters: [90, 10]
scores=0.8470000000000001
Neighborhood algorithm: GaussianNB, parameters: [90, 10]
scores=0.8470000000000001
Neighborhood algorithm: GradientBoostingClassifier, parameters: [59, 0.1471832065983716]
scores=0.9170000000000001
Neighborhood algorithm: RandomForestClassifier, parameters: [67, 3]
scores=0.884
Neighborhood algorithm: LogisticRegression, parameters: [0.04146589299321995, 'l1']
scores=0.808
Neighborhood algorithm: SVC, param

...

scores=0.933
Neighborhood algorithm: GradientBoostingClassifier, parameters: [61, 0.1741530312729914]
scores=0.9219999999999999
Neighborhood algorithm: LogisticRegression, parameters: [5.89705531719389, 'l1']
scores=0.82
Neighborhood algorithm: SVC, parameters: [4.212129272533732, 0.11459059982072832]
scores=0.9269999999999999
Neighborhood algorithm: GaussianNB, parameters: [4.212129272533732, 0.11459059982072832]
scores=0.8470000000000001
Neighborhood algorithm: GradientBoostingClassifier, parameters: [20, 0.2611856671749389]
scores=0.9150000000000003
Neighborhood algorithm: GradientBoostingClassifier, parameters: [72, 0.024841718973101744]
scores=0.897
Neighborhood algorithm: DecisionTreeClassifier, parameters: [3, 8]
scores=0.873
Neighborhood algorithm: MLPClassifier, parameters: [0.01522629652064088, 0.06594809618106513]
scores=0.9359999999999999
Neighborhood algorithm: MLPClassifier, parameters: [0.09061658944798738, 0.09551987395118516]
scores=0.9359999999999999
Neighborhood algo

/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

scores=0.9359999999999999
Neighborhood algorithm: RandomForestClassifier, parameters: [94, 4]
scores=0.9099999999999999
Neighborhood algorithm: DecisionTreeClassifier, parameters: [1, 19]
scores=0.719
Neighborhood algorithm: GaussianNB, parameters: [1, 19]
scores=0.8470000000000001
Neighborhood algorithm: SVC, parameters: [3.2743601583672075, 0.19300662066171445]
scores=0.931
Neighborhood algorithm: GaussianNB, parameters: [3.2743601583672075, 0.19300662066171445]
scores=0.8470000000000001
Neighborhood algorithm: LogisticRegression, parameters: [2.4507137961541563, 'l1']
scores=0.8219999999999998
Neighborhood algorithm: GradientBoostingClassifier, parameters: [94, 0.28895189878984945]
scores=0.925
Neighborhood algorithm: MLPClassifier, parameters: [0.09297226613398743, 0.052367775425155486]
scores=0.9410000000000001
Neighborhood algorithm: LogisticRegression, parameters: [3.981881015969516, 'l1']
scores=0.821
Neighborhood algorithm: MLPClassifier, parameters: [0.0885302277933524, 0.087

/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

scores=0.9380000000000001
Neighborhood algorithm: DecisionTreeClassifier, parameters: [17, 4]
scores=0.901
Neighborhood algorithm: DecisionTreeClassifier, parameters: [19, 4]
scores=0.9039999999999999
Neighborhood algorithm: SVC, parameters: [6.774623861017979, 0.7337069874201356]
scores=0.858
Neighborhood algorithm: RandomForestClassifier, parameters: [35, 15]
scores=0.9289999999999999
Neighborhood algorithm: LogisticRegression, parameters: [9.658777432812414, 'l1']
scores=0.82
Neighborhood algorithm: RandomForestClassifier, parameters: [61, 3]
scores=0.883
Neighborhood algorithm: RandomForestClassifier, parameters: [44, 18]
scores=0.9279999999999999
Neighborhood algorithm: MLPClassifier, parameters: [0.05418373536475805, 0.08795252419897617]
scores=0.932
Neighborhood algorithm: DecisionTreeClassifier, parameters: [18, 4]
scores=0.9019999999999999
Neighborhood algorithm: DecisionTreeClassifier, parameters: [15, 9]
scores=0.9010000000000001
Neighborhood algorithm: GradientBoostingClass